In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - spheroid_disk_decomposition.ipynb
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Specifically look at the spheroid and disk components of the galaxies 
to pare down the sample to only those galaxies which are like the MW in this
way.
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np
import sys, os
import h5py
import glob
import copy
import dill as pickle

## Matplotlib
from matplotlib import pyplot as plt
import matplotlib

sys.path.insert(0,'../../src/')
from tng_dfs import util as putil
from tng_dfs import plot as pplot
from tng_dfs import cutout as pcutout
from tng_dfs import tree as ptree

### Notebook setup
%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [ ]:
# %load ../../src/nb_modules/nb_setup.txt
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','MW_ANALOG_DIR','FIG_DIR_BASE','FITTING_DIR_BASE',
            'RO','VO','ZO','LITTLE_H','MW_MASS_RANGE']
data_dir,mw_analog_dir,fig_dir_base,fitting_dir_base,ro,vo,zo,h,\
    mw_mass_range = putil.parse_config_dict(cdict,keywords)

# MW Analog 
mwsubs,mwsubs_vars = putil.prepare_mwsubs(mw_analog_dir,h=h,
    mw_mass_range=mw_mass_range,return_vars=True,force_mwsubs=False)

# Figure path
local_fig_dir = './fig/'
fig_dir = os.path.join(fig_dir_base, 
    'notebooks/1_analog_samples/3_spheroid_disk_decomposition/')
os.makedirs(local_fig_dir,exist_ok=True)
os.makedirs(fig_dir,exist_ok=True)
show_plots = False

# # Load tree data
# tree_primary_filename = os.path.join(mw_analog_dir,
#     'major_mergers/tree_primaries.pkl')
# with open(tree_primary_filename,'rb') as handle: 
#     tree_primaries = pickle.load(handle)
# tree_major_mergers_filename = os.path.join(mw_analog_dir,
#     'major_mergers/tree_major_mergers.pkl')
# with open(tree_major_mergers_filename,'rb') as handle:
#     tree_major_mergers = pickle.load(handle)
# n_mw = len(tree_primaries)

## Download some supplementary files

In [ ]:
supp_dir = data_dir+'supplementary_data/'
stellar_circs_url = mwsubs_vars['sim']['files']['stellar_circs']
stellar_circs_filename = supp_dir+stellar_circs_url.split('/')[-1]
if not os.path.isfile(stellar_circs_filename):
    putil.get(stellar_circs_url,directory=supp_dir)
else:
    print('Stellar circs file already downloaded.')

In [ ]:
supp_dir = data_dir+'supplementary_data/'
kinematic_decomposition_url = putil.get(mwsubs_vars['sim']['files']['kinematic_decomposition'])['files'][0]
kinematic_decomposition_filename = supp_dir+'kinematic_decomposition_099.hdf5'
if not os.path.isfile(kinematic_decomposition_filename):
    putil.get(kinematic_decomposition_url,directory=supp_dir)
else:
    print('Kinematic decomposition file already downloaded.')

## Examine Milky Way Analog properties

### Stellar circularities file

In [ ]:
fsc = h5py.File(stellar_circs_filename,'r')
snap_key = 'Snapshot_99'
# fsc['Snapshot_99'].keys()
# _keys_load = ['SubfindID',
#               'CircAbove07Frac',
#               'CircAbove07Frac_allstars',
#               'CircAbove07MinusBelowNeg07Frac',
#               'CircAbove07MinusBelowNeg07Frac_allstars',
#               'CircTwiceBelow0Frac',
#               'CircTwiceBelow0Frac_allstars',]
_keys_load = list(fsc[snap_key].keys())
scdata = {}
for key in _keys_load:
    scdata[key] = fsc[snap_key][key][()]
scmask = np.isin(scdata['SubfindID'],mwsubs['id'])
assert np.all( scdata['SubfindID'][scmask] == mwsubs['id'] ) # Make sure actual masking isn't required

In [ ]:
list(fsc[snap_key].keys())

- `CircAbove07Frac` is a proxy for disk fraction
- `CircAbove07MinusBelowNeg07Frac` is a better proxy for disk fraction, since
    it removes the contribution from bar stars
- `CircTwiceBelow0Frac` is a proxy for bulge fraction

In [ ]:
# Some parameters
xkey = 'CircAbove07Frac'
ykey = 'CircAbove07MinusBelowNeg07Frac'
xlim = [-0.1,1.1]
ylim = [-0.1,1.1]
nbin = 10
fig_titles = ['Within twice stellar half-mass radius',
              'All stars']

for i in range(2):
    if i == 1: xkey += '_allstars'; ykey += '_allstars'

    # Make some plots of properties from this file
    fig = plt.figure()
    gs = matplotlib.gridspec.GridSpec(4,4)
    ax = plt.subplot(gs[1:,:3])
    axt = plt.subplot(gs[0,:3])
    axr = plt.subplot(gs[1:,3])

    ax.scatter( scdata[xkey][scmask],
                scdata[ykey][scmask],
        marker='o', facecolor='DodgerBlue', edgecolor='Black', zorder=1)
    ax.set_xlabel(r'$f(\epsilon > 0.7)$]')
    ax.set_ylabel(r'$f(\epsilon > 0.7) - f(\epsilon < -0.7)$')
    ax.set_xlim([xlim[0],xlim[1]])
    ax.set_ylim([ylim[0],ylim[1]])
    ax.plot([-1,2],[-1,2],color='k',linestyle='--',zorder=0)

    # Plot marginal distributions
    axt.hist(scdata[xkey][scmask],bins=nbin,range=(0,1),color='DodgerBlue',
        edgecolor='Black',zorder=1)
    axt.set_xlim([xlim[0],xlim[1]])
    axt.set_ylabel('N')
    axt.tick_params(axis='x', labelbottom=False)

    axr.hist(scdata[ykey][scmask],bins=nbin,range=(0,1),
        orientation='horizontal',color='DodgerBlue',edgecolor='Black',zorder=1)
    axr.set_ylim([ylim[0],ylim[1]])
    axr.set_xlabel('N')
    axr.tick_params(axis='y', labelleft=False)

    fig.suptitle(fig_titles[i])
    fig.show()

In [ ]:
non_p23_mw_analog_ids = [394621, 480802]

# Some parameters
xkey = 'CircTwiceBelow0Frac'
ykey = 'CircAbove07MinusBelowNeg07Frac'
xlim = [-0.1,1.1]
ylim = [-0.1,1.1]
nbin = 10
fig_titles = ['Within twice stellar half-mass radius',
              'All stars']
bulge_cut = [0.45,0.45]
disk_cut = [0.35,0.35]
sc_bulge_disk_masks = []

for i in range(2):
    if i == 1: xkey += '_allstars'; ykey += '_allstars'

    # Make some plots of properties from this file
    fig = plt.figure()
    gs = matplotlib.gridspec.GridSpec(4,4)
    ax = plt.subplot(gs[1:,:3])
    axt = plt.subplot(gs[0,:3])
    axr = plt.subplot(gs[1:,3])

    sc_bulge_disk_mask = (scdata[xkey][scmask] < bulge_cut[i]) & \
                            (scdata[ykey][scmask] > disk_cut[i])
    sc_bulge_disk_masks.append(sc_bulge_disk_mask)

    for j in range(np.sum(sc_bulge_disk_mask)):
        if mwsubs['id'][sc_bulge_disk_mask][j] in non_p23_mw_analog_ids:
            ec, zorder = 'Red', 2
        else:
            ec, zorder = 'Black', 1
        ax.scatter( scdata[xkey][scmask][sc_bulge_disk_mask][j],
                    scdata[ykey][scmask][sc_bulge_disk_mask][j],
            marker='o', facecolor='DodgerBlue', edgecolor=ec, zorder=zorder)
    ax.scatter( scdata[xkey][scmask][~sc_bulge_disk_mask],
                scdata[ykey][scmask][~sc_bulge_disk_mask],
        marker='x', color='Black', zorder=1)
    ax.set_xlabel(r'$2f(\eta < 0)$')
    ax.set_ylabel(r'$f(\eta > 0.7) - f(\eta < -0.7)$')
    ax.set_xlim([xlim[0],xlim[1]])
    ax.set_ylim([ylim[0],ylim[1]])
    ax.axvline(bulge_cut[i], color='k', linestyle='--', zorder=0)
    ax.axhline(disk_cut[i], color='k', linestyle='--', zorder=0)
    # ax.plot([-1,2],[-1,2],color='k',linestyle='--',zorder=0)

    # Annotate each point
    # for j in range(len(mwsubs)):
    #     ax.annotate(str(mwsubs['id'][j]), xy=(scdata[xkey][scmask][j], 
    #         scdata[ykey][scmask][j]), xycoords='data', fontsize=4,
    #         color='Black')

    # Plot marginal distributions
    axt.hist(scdata[xkey][scmask],bins=nbin,range=(0,1),color='DodgerBlue',
        edgecolor='Black',zorder=1)
    axt.axvline(bulge_cut[i], color='k', linestyle='--', zorder=0)
    axt.set_xlim([xlim[0],xlim[1]])
    axt.set_ylabel('N')
    axt.tick_params(axis='x', labelbottom=False)

    axr.hist(scdata[ykey][scmask],bins=nbin,range=(0,1),
        orientation='horizontal',color='DodgerBlue',edgecolor='Black',zorder=1)
    axr.axhline(disk_cut[i], color='k', linestyle='--', zorder=0)
    axr.set_ylim([ylim[0],ylim[1]])
    axr.set_xlabel('N')
    axr.tick_params(axis='y', labelleft=False)

    # Circle showing the Milky Way
    mw_circle = plt.Circle((0.3,0.7), 0.05 ,zorder=0, 
        alpha=0.25, edgecolor='none', facecolor='Black')
    ax.add_artist(mw_circle)

    if i > 0:
        fig.suptitle(fig_titles[i])
    fig.show()

    # if i == 0:
    sc_bulge_disk_mask = (scdata[xkey][scmask] < bulge_cut[i]) & \
                            (scdata[ykey][scmask] > disk_cut[i])
    sc_bulge_disk_masks.append(sc_bulge_disk_mask)


In [ ]:
print('Twice stellar half mass radius included:')
print('Number in: ',np.sum(sc_bulge_disk_masks[0]))
print( mwsubs['id'][sc_bulge_disk_masks[0]] )
print('Excluded:')
print( mwsubs['id'][~sc_bulge_disk_masks[0]] )
print('\n\nAll stars included:')
print('Number in: ',np.sum(sc_bulge_disk_masks[1]))
print( mwsubs['id'][sc_bulge_disk_masks[1]] )
print('Excluded:')
print( mwsubs['id'][~sc_bulge_disk_masks[1]] ) 
print('\n\nMasks are the same: '+np.all(sc_bulge_disk_masks[0]==sc_bulge_disk_masks[1]).astype(str))
sc_bulge_disk_mask = sc_bulge_disk_masks[0]

# Save the mask
mask_filename = os.path.join(mw_analog_dir, 'masks/', 'bulge_disk_fraction_z0sid.npy')
np.save( mask_filename, mwsubs['id'][sc_bulge_disk_mask] )

### Kinematic decompositions file

In [ ]:
fkd = h5py.File(kinematic_decomposition_filename,'r')
_keys_load = list(fkd.keys())
print(_keys_load)
_class1_keys = list(fkd['class1_allstars'].keys())
print(_class1_keys)
_class2_keys = list(fkd['class2_allstars'].keys())
print(_class2_keys)

# _keys_load.remove('Header')
# kddata = {}
# for key in _keys_load:
#     kddata[key] = fkd[key][()]

kdmask = np.isin(fkd['SubhaloID'],mwsubs['id'])

In [ ]:
# Some parameters
xlim = [-0.1,1.1]
ylim = [-0.1,1.1]
nbin = 10
method_keys = ['class1_1re',
               'class1_30kpc',
               'class1_3re',
               'class1_allstars',]
fig_titles = ['Class 1, Within 1 stellar half-mass radius',
              'Class 1, Within 30 kpc',
              'Class 1, Within 3 stellar half-mass radii',
              'Class 1, All stars']

for i in range(len(method_keys)):

    # Data
    fbulge = fkd[method_keys[i]]['Bulge'][kdmask]
    fdisk = fkd[method_keys[i]]['Disks'][kdmask]
    star_mass = fkd[method_keys[i]]['StellarMass'][kdmask]

    # Make some plots of properties from this file
    fig = plt.figure()
    gs = matplotlib.gridspec.GridSpec(4,4)
    ax = plt.subplot(gs[1:,:3])
    axt = plt.subplot(gs[0,:3])
    axr = plt.subplot(gs[1:,3])

    pts1 = ax.scatter(fbulge[sc_bulge_disk_mask], fdisk[sc_bulge_disk_mask],
        marker='o', c=star_mass[sc_bulge_disk_mask]/1e10,
        edgecolor='Black', cmap='rainbow', zorder=1)
    pts2 = ax.scatter(fbulge[~sc_bulge_disk_mask], fdisk[~sc_bulge_disk_mask],
        marker='x', c=star_mass[~sc_bulge_disk_mask]/1e10,
        edgecolor='Black', cmap='rainbow', zorder=1)
    ax.set_xlabel(r'$f_{bulge}(r)$')
    ax.set_ylabel(r'$f_{disk}(r)$')
    constant_fracs = [0.5,0.75,1]
    for j in range(len(constant_fracs)):
        ax.plot([constant_fracs[j],0], [0,constant_fracs[j]], 
            linestyle='dashed', color='Black')
    ax.set_xlim([xlim[0],xlim[1]])
    ax.set_ylim([ylim[0],ylim[1]])
    # ax.plot([-1,2],[-1,2],color='k',linestyle='--',zorder=0)

    # Make a colorbar in the top-right corner
    cax = fig.add_axes([0.75, 0.8, 0.15, 0.02])
    cbar = fig.colorbar(pts1, cax=cax, orientation='horizontal')
    cbar.set_label(r'$M_{\star}$ [$10^{10} \mathrm{M}_{\odot}$]')

    # Plot marginal distributions
    axt.hist(fbulge,bins=nbin,color='DodgerBlue',range=(0,1),
        edgecolor='Black',zorder=1)
    axt.set_xlim([xlim[0],xlim[1]])
    axt.set_ylabel('N')
    axt.tick_params(axis='x', labelbottom=False)

    axr.hist(fdisk,bins=nbin,range=(0,1),
        orientation='horizontal',color='DodgerBlue',edgecolor='Black',zorder=1)
    axr.set_ylim([ylim[0],ylim[1]])
    axr.set_xlabel('N')
    axr.tick_params(axis='y', labelleft=False)

    fig.suptitle(fig_titles[i])
    fig.show()

In [ ]:
# Some parameters
xlim = [-0.1,1.1]
ylim = [-0.1,1.1]
nbin = 10
method_keys = ['class2_1re',
               'class2_30kpc',
               'class2_3re',
               'class2_allstars',]
fig_titles = ['Class 2, Within 1 stellar half-mass radius',
              'Class 2, Within 30 kpc',
              'Class 2, Within 3 stellar half-mass radii',
              'Class 2, All stars']

for i in range(len(method_keys)):

    # Data
    fbulge = fkd[method_keys[i]]['Bulge'][kdmask]
    fdisk = fkd[method_keys[i]]['Disks'][kdmask]
    star_mass = fkd[method_keys[i]]['StellarMass'][kdmask]

    # Make some plots of properties from this file
    fig = plt.figure()
    gs = matplotlib.gridspec.GridSpec(4,4)
    ax = plt.subplot(gs[1:,:3])
    axt = plt.subplot(gs[0,:3])
    axr = plt.subplot(gs[1:,3])

    pts1 = ax.scatter(fbulge[sc_bulge_disk_mask], fdisk[sc_bulge_disk_mask],
        marker='o', c=star_mass[sc_bulge_disk_mask]/1e10,
        edgecolor='Black', cmap='rainbow', zorder=1)
    pts2 = ax.scatter(fbulge[~sc_bulge_disk_mask], fdisk[~sc_bulge_disk_mask],
        marker='x', c=star_mass[~sc_bulge_disk_mask]/1e10,
        edgecolor='Black', cmap='rainbow', zorder=1)
    ax.set_xlabel(r'$f_{bulge}(r)$')
    ax.set_ylabel(r'$f_{disk}(r)$')
    constant_fracs = [0.5,0.75,1]
    for j in range(len(constant_fracs)):
        ax.plot([constant_fracs[j],0], [0,constant_fracs[j]], 
            linestyle='dashed', color='Black')
    ax.set_xlim([xlim[0],xlim[1]])
    ax.set_ylim([ylim[0],ylim[1]])
    # ax.plot([-1,2],[-1,2],color='k',linestyle='--',zorder=0)

    # Make a colorbar in the top-right corner
    cax = fig.add_axes([0.75, 0.8, 0.15, 0.02])
    cbar = fig.colorbar(pts1, cax=cax, orientation='horizontal')
    cbar.set_label(r'$M_{\star}$ [$10^{10} \mathrm{M}_{\odot}$]')

    # Plot marginal distributions
    axt.hist(fbulge,bins=nbin,color='DodgerBlue',range=(0,1),
        edgecolor='Black',zorder=1)
    #axt.set_xlim([xlim[0],xlim[1]])
    axt.set_ylabel('N')
    axt.tick_params(axis='x', labelbottom=False)
    axt.set_xlim([xlim[0],xlim[1]])

    axr.hist(fdisk,bins=nbin,range=(0,1),
        orientation='horizontal',color='DodgerBlue',edgecolor='Black',zorder=1)
    #axr.set_ylim([ylim[0],ylim[1]])
    axr.set_xlabel('N')
    axr.tick_params(axis='y', labelleft=False)
    axr.set_ylim([ylim[0],ylim[1]])

    fig.suptitle(fig_titles[i])
    fig.show()

### Try and do this ourselves by just using the standard J/Jcirc and E/Enorm bounds

In [ ]:
verbose = True
show_plots = True

fbulge = np.zeros(n_mw)
fthin = np.zeros(n_mw)
fthick = np.zeros(n_mw)
fhalo = np.zeros(n_mw)

for i in range(n_mw):
    # if i < 10: continue
    if verbose: print(f'Doing MW {i+1}/{n_mw}', end='\r')

    # Get the primary
    snapnum = mwsubs[i]['snap']
    z0_sid = mwsubs[i]['id']
    primary_filename = putil.get_cutout_filename(mw_analog_dir,
        snapnum, z0_sid)
    co = pcutout.TNGCutout(primary_filename)
    co.center_and_rectify()
    
    # Get properties, energy, angular momentum
    orbs = co.get_orbs('PartType4')
    rs = orbs.r().value
    vels = co.get_velocities('PartType4', physical=True)
    pot = co.get_potential_energy('PartType4', physical=True)
    kin = 0.5*np.sum(np.square(vels),axis=1)
    E = (pot+kin)
    J,Jz,Jp = co.get_J_Jz_Jp('PartType4',physical=True)
    co.get_E_Jcirc_spline('PartType4',angmom='J')
    Jcirc = co.Jcirc(E)
    Enorm = E/np.abs(E).max()
    Jz_Jcirc = Jz / Jcirc
    Jp_Jcirc = Jp / Jcirc
    
    # Mask the halo using these quantities
    Jz_Jcirc_spheroid_bound = 0.5
    Jz_Jcirc_disk_bound = 0.8
    Enorm_bulge_bound = -0.75

    bulge_mask = (Jz_Jcirc < Jz_Jcirc_spheroid_bound) &\
                (Enorm < Enorm_bulge_bound)
    halo_mask = (Jz_Jcirc < Jz_Jcirc_spheroid_bound) &\
                (Enorm > Enorm_bulge_bound)
    thin_mask = (Jz_Jcirc > Jz_Jcirc_disk_bound)
    thick_mask = (Jz_Jcirc < Jz_Jcirc_disk_bound) &\
                (Jz_Jcirc > Jz_Jcirc_spheroid_bound)
    
    fbulge[i] = np.sum(bulge_mask) / len(bulge_mask)
    fthin[i] = np.sum(thin_mask) / len(thin_mask)
    fthick[i] = np.sum(thick_mask) / len(thick_mask)
    fhalo = np.sum(halo_mask) / len(halo_mask)

fdisk = fthin + fthick

with open('data/Enorm_Jcirc_fractions.pkl','wb') as handle:
    about = ('indices 0-3 in this array are fbulge, fthin, fthick, fhalo,'
             ' defined according to standard E/Enorm - Jz/Jcirc decomposition')
    pickle.dump([fbulge,fthin,fthick,fhalo,about],handle)

In [ ]:
xlim = [-0.1,1.1]
ylim = [-0.1,1.1]
nbin = 10

# Make some plots of properties from this file
fig = plt.figure(figsize=(10,10))
gs = matplotlib.gridspec.GridSpec(4,4)
ax = plt.subplot(gs[1:,:3])
axt = plt.subplot(gs[0,:3])
axr = plt.subplot(gs[1:,3])

pts = ax.scatter(fbulge[sc_bulge_disk_mask], fdisk[sc_bulge_disk_mask],
    marker='o', edgecolor='Black', cmap='rainbow', zorder=1)
pts = ax.scatter(fbulge[~sc_bulge_disk_mask], fdisk[~sc_bulge_disk_mask],
    marker='x', edgecolor='Black', cmap='rainbow', zorder=1)
ax.set_xlabel(r'$f_{bulge}(r)$')
ax.set_ylabel(r'$f_{disk}(r)$')
constant_fracs = [0.5,0.75,1]
for j in range(len(constant_fracs)):
    ax.plot([constant_fracs[j],0], [0,constant_fracs[j]], 
        linestyle='dashed', color='Black')
ax.set_xlim([xlim[0],xlim[1]])
ax.set_ylim([ylim[0],ylim[1]])
for i in range(n_mw):
    z0_sid = mwsubs[i]['id']
    ax.annotate(str(z0_sid), xy=(fbulge[i],fdisk[i]), fontsize=6, zorder=2,
        color='Black', ha='left', va='center', alpha=0.75)
# ax.plot([-1,2],[-1,2],color='k',linestyle='--',zorder=0)

# # Make a colorbar in the top-right corner
# cax = fig.add_axes([0.75, 0.8, 0.15, 0.02])
# cbar = fig.colorbar(pts, cax=cax, orientation='horizontal')
# cbar.set_label(r'$M_{\star}$ [$10^{10} \mathrm{M}_{\odot}$]')

# Plot marginal distributions
axt.hist(fbulge,bins=nbin,color='DodgerBlue',range=(0,1),
    edgecolor='Black',zorder=1)
#axt.set_xlim([xlim[0],xlim[1]])
axt.set_ylabel('N')
axt.tick_params(axis='x', labelbottom=False)
axt.set_xlim([xlim[0],xlim[1]])

axr.hist(fdisk,bins=nbin,range=(0,1),
    orientation='horizontal',color='DodgerBlue',edgecolor='Black',zorder=1)
#axr.set_ylim([ylim[0],ylim[1]])
axr.set_xlabel('N')
axr.tick_params(axis='y', labelleft=False)
axr.set_ylim([ylim[0],ylim[1]])

fig.show()

### Calculate the mass fraction in ordered rotation, $\kappa$

In [ ]:
verbose = True
kappas = np.zeros(n_mw)

for i in range(n_mw):
    # if i > 2: continue
    if verbose: print(f'Getting MW {i+1}/{n_mw}', end='\r')

    # Get the primary
    snapnum = mwsubs[i]['snap']
    z0_sid = mwsubs[i]['id']
    primary_filename = putil.get_cutout_filename(mw_analog_dir,
        snapnum, z0_sid)
    co = pcutout.TNGCutout(primary_filename)
    co.center_and_rectify()

    orbs = co.get_orbs('stars')
    r = orbs.r().value
    rhm = co.get_half_mass_radius('stars').value
    mask = (r < 2*rhm)
    masses = co.get_masses('stars', physical=True).value
    vR, vT, vz = orbs.vR().value, orbs.vT().value, orbs.vz().value
    vmag = np.sqrt(vR**2 + vT**2 + vz**2)

    kappas[i] = np.sum(masses[mask] * vT[mask]**2)/\
        np.sum(masses[mask] * vmag[mask]**2)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

lim = [0.2,1.]
nb = 16
ax.hist(kappas, bins=nb, range=lim, histtype='step', edgecolor='Black', 
    linewidth=4., zorder=1)
ax.hist(kappas[sc_bulge_disk_mask], bins=nb, range=lim, histtype='step', 
    linewidth=1., edgecolor='DodgerBlue', zorder=2)
ax.hist(kappas[~sc_bulge_disk_mask], bins=nb, range=lim, histtype='step', 
    linewidth=1., edgecolor='Red', zorder=3, linestyle='dashed')

ax.set_xlabel(r'$\kappa$')
ax.set_ylabel('N')

fig.show()